# News Project
> The goal of this project is to create a newspaper like app for dialy news. 

I want a facts first, unbiased version of the news. I dont want to endlessly scroll news and memes all day. 

In [1]:
import feedparser
import time
from datetime import datetime

def fetch_google_news(country='US', language='en', topic=None, query=None):
    base_url = f"https://news.google.com/rss?hl={language}&gl={country}&ceid={country}:{language}"
    
    if topic:
        base_url += f"&topic={topic}"
    elif query:
        base_url = f"https://news.google.com/rss/search?q={query}&hl={language}&gl={country}&ceid={country}:{language}"

    print(f"Fetching news from: {base_url}")
    
    feed = feedparser.parse(base_url)
    
    articles = []
    for entry in feed.entries:
        articles.append({
            'title': entry.title,
            'link': entry.link,
            'published': entry.published,
            'summary': entry.summary,
            'source': entry.source.href if hasattr(entry, 'source') else 'Unknown'
        })
    
    return articles


# Example usage
LANGUAGE_CODE = 'en'
COUNTRY_CODE = 'US'

# us_news = fetch_google_news(country=COUNTRY_CODE, language=LANGUAGE_CODE)
world_news = fetch_google_news(language=LANGUAGE_CODE, topic='world')

for article in world_news:
    print(f"Title: {article['title']}")
    print(f"Link: {article['link']}")
    print(f"Published: {article['published']}")
    print(f"Summary: {article['summary']}")
    print(f"Source: {article['source']}\n")
    



Fetching news from: https://news.google.com/rss?hl=en&gl=US&ceid=US:en&topic=world
Title: Israeli forces kill 67 Palestinians seeking aid in northern Gaza, Hamas-run ministry says - BBC
Link: https://news.google.com/rss/articles/CBMiWkFVX3lxTE54RmJWVEpaU1JlbFNzYXZyQ1ZqZ2RzbHp5VXp3M0hXc0U1Y2djZkVMZDFraVR1VDRDOUZLdHV0ODdpQ2xPSnN4cUx4RDZUV0t5MjhOSllNSGRoUdIBX0FVX3lxTFBVZGxQcGdKSHJwdlg1a0ZkeDBLRmNrQldkZ2RtWlJCVjk1aUZISkt2S3JKd1IyZUpGNFpEeU52VTl0SktHRjJlQjdDNmVWdnZvUFdPMFBkU1RuaVNpaWdF?oc=5
Published: Sun, 20 Jul 2025 21:11:08 GMT
Summary: <ol><li><a href="https://news.google.com/rss/articles/CBMiWkFVX3lxTE54RmJWVEpaU1JlbFNzYXZyQ1ZqZ2RzbHp5VXp3M0hXc0U1Y2djZkVMZDFraVR1VDRDOUZLdHV0ODdpQ2xPSnN4cUx4RDZUV0t5MjhOSllNSGRoUdIBX0FVX3lxTFBVZGxQcGdKSHJwdlg1a0ZkeDBLRmNrQldkZ2RtWlJCVjk1aUZISkt2S3JKd1IyZUpGNFpEeU52VTl0SktHRjJlQjdDNmVWdnZvUFdPMFBkU1RuaVNpaWdF?oc=5" target="_blank">Israeli forces kill 67 Palestinians seeking aid in northern Gaza, Hamas-run ministry says</a>&nbsp;&nbsp;<font color="#6f6f6f"

In [ ]:
from bs4 import BeautifulSoup
import requests
import json

session = requests.Session()

# List to holder articles [(url, content), ...]
articles = []

def get_real_url(google_news_url):
    try:
        # Step 1: Get the Google News page
        resp = requests.get(google_news_url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # Step 2: Extract the special data
        cwiz = soup.select_one('c-wiz[data-p]')
        if not cwiz:
            return None
            
        data = cwiz.get('data-p')
        obj = json.loads(data.replace('%.@.', '["garturlreq",'))
        
        # Step 3: Make POST request to Google's API
        payload = {
            'f.req': json.dumps([[['Fbv4je', json.dumps(obj[:-6] + obj[-2:]), 'null', 'generic']]])
        }
        
        headers = {
            'content-type': 'application/x-www-form-urlencoded;charset=UTF-8',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.post("https://news.google.com/_/DotsSplashUi/data/batchexecute", 
                               headers=headers, data=payload)
        
        # Step 4: Parse the response to get the real URL
        array_string = json.loads(response.text.replace(")]}'", ""))[0][2]
        article_url = json.loads(array_string)[1]
        
        return article_url
        
    except Exception as e:
        print(f"Error getting real URL: {e}")
        return None

def scrape_article(google_news_link):
    try:
        # Scrape
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

        # redirect_response = requests.get(google_link, headers=headers, allow_redirects=True, timeout=30)
        real_news_url = get_real_url(google_news_link)
        print(f"Redirected URL: {real_news_url}")
        response = requests.get(real_news_url, headers=headers, allow_redirects=True, timeout=30) 
        time.sleep(2)
        soup = BeautifulSoup(response.content, 'html.parser')
        article = (soup.find('article') or 
                  soup.find('div', class_='article-content') or
                  soup.find('div', class_='FITT_Article_main__body') or
                  soup.find('div', class_='theme-e') or
                  soup.find('main'))
        
        # If no article try bypassing
        if not article:
            # bypass_link = f"https://archive.is/latest/{real_news_url}"
            bypass_link = f"https://web.archive.org/web/{real_news_url}"
            print(f"Bypassing with: {bypass_link}")
            response = requests.get(bypass_link, headers=headers, allow_redirects=False, timeout=30)
            if response.status_code in [301, 302, 303, 307, 308]:
                redirect_url = response.headers.get('Location')
                if redirect_url and 'web.archive' in redirect_url:
                    time.sleep(60)
                    print(f"Following archive.is redirect to: {redirect_url}")
                    response = session.get(redirect_url, headers=headers, timeout=30)
                else:
                    time.sleep(10)
                    print(f"Blocked redirect to: {redirect_url}")
            print(f"Response status code: {response.status_code}")
            print(f"Response URL: {response.url}")
            time.sleep(2)
            soup = BeautifulSoup(response.content, 'html.parser')
            article = (soup.find('article') or 
                  soup.find('div', class_='article-content') or
                  soup.find('div', class_='FITT_Article_main__body') or
                  soup.find('div', class_='theme-e') or
                  soup.find('main'))
                  

        return article
    except Exception as e:
        print(f"Error scraping {google_news_link}: {e}")
        return ""


# Scrape from sites to get articles
for article in world_news:
    full_content = scrape_article(article['link'])
    # Extract text from BeautifulSoup Tag before slicing
    if hasattr(full_content, 'get_text'):
        content_text = full_content.get_text()
    else:
        content_text = str(full_content)
    articles.append((article['link'], content_text[:100]))  # First 100 characters for testing
    print(f"Full content: {content_text}\n")


Redirected URL: https://www.bbc.com/news/articles/c8rp62480r3o
Full content: Israeli forces kill 67 Palestinians seeking aid in northern Gaza, Hamas-run ministry says5 hours agoShareSaveYolande KnellMiddle East correspondentReporting fromJerusalemJack BurgessBBC NewsShareSaveReutersPalestinians pictured carrying aid supplies in Beit Lahia, northern Gaza, on SundayThe Israeli military has killed at least 67 people waiting for UN aid lorries in northern Gaza, the territory's Hamas-run health ministry says.The UN World Food Programme said its 25-truck convoy "encountered massive crowds of hungry civilians which came under gunfire", soon after it crossed from Israel and cleared checkpoints.The Israel Defense Forces (IDF) said that it had "fired warning shots" to remove "an immediate threat". It disputed the number of reported deaths.On Saturday the ministry warned that extreme hunger was increasing in Gaza and growing numbers of people were arriving at its facilities "in a state of extreme

KeyboardInterrupt: 